## 1.最大线段重合问题
给定很多线段，每个线段都有两个数组[start,end]，表示线段开始、结束位置，且闭区间。
* 规定1：线段的开始和结束位置一定都是整数值
* 规定2：线段重合区域的长度必须≥1
* 返回：线段最多重合区域中，包含了几条线段

In [2]:
# 笨办法：找到开始的最小值min，结束的最大值max。所有重合区域必在[min,max]
# 分别计算 min+0.5, min+1.5 ... ,max - 0.5 出现在多少重合区域内。
# 时间复杂度O((max-min)*N)

def maxCover1(lines):
    if not lines or not len(lines[0]):
        return 0
    min_start = lines[0][0]
    max_end = lines[0][1]
    for i in range(1, len(lines)):
        min_start = min(min_start, lines[i][0])
        max_end = min(max_end, lines[i][1])
    cover = 0
    for p in range(min_start, max_end):
        p += 0.5
        cur = 0
        for i in range(len(lines)):
            if lines[i][0] < p and lines[i][1] > p:
                cur += 1
        cover = max(cover, cur)
    return cover

In [14]:
'''
思路：重合区域的左边界，必是线段的左边界。O(N*logN)
所以遍历线段，假设当前线段的开始位置为重合区域的左边界，求有多少线段穿过了这个左边界
1.线段按开始位置，从小到大排序（只按开始位置排序，当开始位置相同时，结束位置先遇到的是大是小不一定）
2.准备一个小根堆。一个线段[min,max]，先将小根堆内≤min的数全部弹出，然后把max压入，此时小根堆剩余数即为【以开始位置作为重合区域，有几条线段可以穿过(包括自己)】
3.遍历(排好序后的)所有线段，重复2操作，选取答案最大的。
''' 
import heapq
def maxCover2(m):
    if not m or not len(m[0]):
        return 0 
    lines = sorted(m, key=lambda x: x[0])
    heap = list()
    ans = 0
    for i in range(len(lines)):
        while heap and heap[0] <= lines[i][0]:
            heapq.heappop(heap)
        heapq.heappush(heap, lines[i][1])
        ans = max(ans, len(heap))
    return ans



In [18]:
arr = [[1,4],[2,5],[6,9],[3,4],[4,5],[1,9]]
maxCover1(arr)

4

## 2.增强堆

In [58]:
''' 为什么要手写堆：自带heap内部没有反向索引表,即各个对象放在了哪里。

1. 如果改了某一个对象，需要heapify或heapInsert重新调整堆结构
2. 在调整前， 需要经过O(N)代价找到这个对象， ！！因为没有反向索引表！！
'''
class HeapEnhanced: #小根堆
    def __init__(self): #若heap里存的是非基础类型，在对象类里自定义比较函数__lt__,__le__,__eq__
        self.heap = list()
        self.indexMap = dict()
        self.heapSize = 0
        
    def isEmpty(self):
        return self.heapSize == 0
    
    def size(self):
        return self.heapSize
    
    def contains(self, obj):
        return obj in self.indexMap
    
    def peek(self):
        return heap[0] #报错为操作问题
    
    def swap(self, i, j): #indexMap也要换
        oi = self.heap[i]
        oj = self.head[j]
        self.heap[j] = oi
        self.heap[i] = oj
        self.indexMap[oi] = j
        self.indexMap[oj] = j
        
    def heapInsert(self, index):
        while index > 0 and self.heap[index] < self.heap[(index - 1)//2]:
            self.swap(index, (index-1)//2)
            index = (index-1)//2
            
    def heapify(self, index):
        left = index * 2 + 1
        while left < self.heapSize:
            best = left
            if (left + 1) < self.heapSize and self.heap[left] > self.heap[left+1]:
                best = left + 1
            if self.heap[best] >= self.heap[index]:
                break
            else:
                self.swap(best, index)
                index = best
                left = index * 2 + 1
    
    def push(self, obj):
        #如果有重复值，加入必须非基础类型；不然indexMap中会覆盖。
        self.heap.append(obj)
        self.indexMap[obj] = self.heapSize
        self.heapInsert(self.heapSize)
        self.heapSize += 1
        
    def pop(self):
        ans = self.heap[0] #可能报错
        self.swap(0, self.heapSize - 1)
        self.indexMap.pop(ans)
        self.heap.pop() #remove last one 
        self.heapSize -= 1
        self.heapify(0)
        return ans
        
    def remove(self, obj): #自定义的好处，通过indexMap快速查询到obj位置
        if obj in self.indexMap:
            index = self.indexMap.pop(obj)
            replace = self.heap.pop()
            self.heapSize -= 1
            if obj != replace:
                self.heap[index] = replace
                self.indexMap[replace] = index
                self.resign(replace)
    
    def resign(self, obj):
        if obj in self.indexMap:
            index = self.indexMap[obj]
            self.heapInsert(index)
            self.heapify(index)
            
        

In [49]:
class Obj:
    def __init__(self, id, score):
        self.id = id
        self.score = score
    
    def __lt__(self, other):
        return self.id < other.id if self.score == other.score else self.score < other.score
    
    def __le__(self,other):
        return self.score <= other.score
    
    def __eq__(self, other):
        return self.id == other.id and self.score == other.score
    
    def __hash__(self):
        return hash(id(self))

In [50]:
a = Obj(1,2)
b = Obj(1,2)

In [51]:
c = dict()

In [52]:
c[a] = 1
c[b] = 25

In [53]:
c

{<__main__.Obj at 0x105f150f0>: 1, <__main__.Obj at 0x105f15be0>: 25}

In [56]:
c.pop(a)

1

In [57]:
c

{<__main__.Obj at 0x105f15be0>: 25}

## 3.手写堆练习
3.1 整数数组arr，布尔数组op。两个数组一定等长，假设长度为N，arr[i]表示客户编号，op[i]表示客户操作。op[i]==T代表用户购买了一件商品，op[i]==False代表用户退了一件商品。
* arr = [3,3,1,2,1,2,5...]
* op  = [T,T,T,T,F,T,F...]
* 表示为 3用户购买一件商品，3用户购买一件商品，1用户购买一件商品，2用户购买一件商品，1用户退货一件商品，2用户购买一件商品，5用户退货一件商品

3.2 作为电商负责人，想在每个事件到来的时候，都给购买次数最多的前K用户颁奖。所以在每个事件发生后，都需要一个得奖名单
* 如果某个时间点，用户购买量为0，而又发生了退货事件，忽略(失效操作)
* 购买+1，退货-1
* 每次都是最多K个用户得奖，K也为传入参数；如果不够K个人，那以不够的情况输出结果
* 得奖系统分为得奖区和候选区，任何用户购买数>0，那用户一定在这两个区域中的一个
* 购买数最大的前K个用户进入得奖区，如果没有K个用户在得奖区，那么新购买的用户直接进入得奖区
* 如果购买数不足以进入得奖区的用户，进入候选区(购买数并未超过第K个用户，或购买数相等，但进入时间点晚于他)
* 如果候选区用户购买数超过(严格大于)得奖区购买数最少的用户，则进入得奖区(替换原来购买最少的那个)
* 如果得奖区购买数最少的用户有多个，此时最早进入得奖区的被替换
* 如果候选区购买数最多的用户有多个，此时最早进入候选区的用户替换进入得奖区

3.3 候选区和得奖区是两套时间，因为用户只会进入其中一个区域，所以只会有一个区域的时间
* 从得奖区出来进入候选区的用户，得奖时间删除，进入候选区的时间就是当前事件的时间(可以理解为arr[i]和op[i]的i)
* 从候选区进入得奖区的用户，候选区时间删除，进入得奖区的时间就是当前事件的时间(可以理解为arr[i]和op[i]的i)
* 如果某用户购买数=0，不管在哪个区域都会离开，区域时间删除。再次发生购买行为，回到某个区域，进入区域的时间重记

In [ ]:
# 暴力解

from functools import cmp_to_key
def compareCands(x, y): #定义谁先从候选区弹出
    if x.buy == y.buy:
        return x.enterTime - y.enterTime #谁早谁放前面
    else:
        return y.buy - x.buy #x多于y时放前面，即谁多谁放前面
    
def compareDaddy(x, y): #定义谁先从得奖区弹出
    if x.buy == y.buy:
        return x.enterTime - y.enterTime #谁早谁放前面
    else:
        return x.buy - y.buy #y多于x时放前面，即谁少谁放前面

class Customer:
    def __init__(self, id, buy, enterTime=0):
        self.id = id
        self.buy = buy
        self.enterTime = enterTime

def move(cands, daddy, k, time):
    #候选区空，不牵扯替换问题
    if not cands:
        return
    #候选区不为空，得奖区小于k（即发生的是退货行为，得奖区刚刚被删掉一个且只删了一个）
    if len(daddy) < k:
        c = cands.pop(0) #候选区拿出第一个用户
        c.enterTime = time
        daddy.append(c) #放入得奖区
    #得奖区满的，候选区有用户
    else:
        if cands[0].buy > daddy[0].buy: #因为严格大于得奖区的最少的用户才替换
            oldDaddy = daddy.pop(0) #得奖区移除第一个用户
            newDaddy = cands.pop(0) #候选区拿出第一个用户
            oldDaddy.enterTime = time
            newDaddy.enterTime = time
            daddy.append(newDaddy)
            cands.append(oldDaddy)
            #排序放在下一个事件发生时
    return

def cleanZeroBuy(arr):
    noZero = list()
    for c in arr:
        if c.buy != 0:
            noZero.append(c)
    arr[:] = noZero[:]
    return
            
def getCurAns(daddy):
    ans = list()
    for c in daddy:
        ans.append(c.id)
    return ans
    
def compareTest(arr, op, k):
    customer_map = dict()
    cands = list()
    daddy = list()
    ans = list()
    for i in range(len(arr)):
        customer = arr[i]
        buyOrRefund = op[i]
        #发生的是退货，且用户不在map里(0购买数)，记为失效
        if not buyOrRefund and customer not in customer_map: 
            ans.append(getCurAns(daddy))
            continue
        #要么购买数是0，发生的是购买事件；要么购买数>0，发生的是购买事件；要么购买数>0，发生的是退货事件
        #如果购买数是0，先加入map（因为购买数、事件时间后面都要重记）
        if customer not in customer_map:
            customer_map[customer] = Customer(customer, 0, 0)
        # 用户一定在map里，更新购买数
        c = customer_map[customer]
        if buyOrRefund:
            c.buy += 1
        else:
            c.buy -= 1
        # 用户购买数为0，先从map中删除
        if c.buy == 0:
            customer_map.pop(customer)
        
        #新用户，且发生购买（购买数不为0）
        if c not in cands and c not in daddy:
            #得奖区不满k个，直接进入得奖区
            if len(daddy) < k:
                c.enterTime = i
                daddy.append(c)
            #得奖区满了，进入候选区，是否进入得奖区后面再说
            else:
                c.enterTime = i
                cands.append(C)
        cleanZeroBuy(cands)
        cleanZeroBuy(daddy)
        candsKey = cmp_to_key(compareCands)
        daddyKey = cmp_to_key(compareDaddy)
        cands = sorted(cands, key=candsKey) #购买数多且早的放前面
        daddy = sorted(daddy, key=daddyKey) #购买数少且早的放前面
        move(cands, daddy, k, i)
        ans.append(getCurAns(daddy))
    return ans
        
        

In [ ]:
# 优化版本, 待重写。重点在于使用小根堆(自定义)增、删、重排用户！
class WhosYourDaddy:
    def __init__(self, limit):
        self.limit = limit
        self.customers = dict()
        self.candsHeap = HeapEnhanced() #python需要在自定义类(Customer)中自定义排序方式
        self.daddyHeap = HeapEnhanced()
    
    def operate(time, id, buyOrRefund):
        

def topK(arr, op, k):
    ans = list()
    whoDaddies = WhosYourDaddy(k)
    for i in range(len(arr)):
        whoDaddies.operate(i, arr[i], op[i])
        ans.append(whoDaddies.getDaddies())
    return ans



In [83]:
a = [3,4]
from functools import cmp_to_key
def compare(a, b):
    return ~(a > b)
k = cmp_to_key(compare)
sorted(a, key=k)

[4, 3]

In [76]:
test = lambda x,y: x-y

In [65]:
a = T(2)
b = T(1)

In [66]:
a > b

True

In [70]:
not 0

True